### 1 -Importons les différentes librairies 

In [3]:
#!pip install tensorflow-gpu==2.0.0beta1
!pip install pygeohash 
!pip install haversine
!pip intall pickle

ERROR: unknown command "intall" - maybe you meant "install"


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import pytz
import pygeohash as gh
from haversine import haversine
import time
import pickle as cPickle
import glob
import json
import warnings
warnings.filterwarnings("ignore")

geohash_prec = 5 # the geo-hash level to define a region R

####*2 - Définnisons une classe weather qui utilise les données météorologiques pour créer des vecteurs features*

In [5]:
class weather:
    date = ''
    temp = 0.0
    windchill = 0.0
    humid = 0.0
    pressure= 0.0
    visib = 0.0
    windspeed = 0.0
    winddir = ''
    precipitation = 0.0
    events = ''
    condition = ''
    
    def __init__(self, date, temp, windchill, humid, pressure, visib, windspeed, winddir, 
                 precipitation, events, condition, zone):
        self.date = datetime.strptime(date, '%Y-%m-%d %I:%M:%S %p')
        self.date = self.date.replace(tzinfo=pytz.timezone(zone))
        self.temp = float(temp)
        self.windchill = float(windchill)
        self.humid = float(humid)
        self.pressure = float(pressure)
        self.visib = float(visib)
        self.windspeed = float(windspeed)
        self.winddir = winddir
        self.precipitation = float(precipitation)
        self.events = events
        self.condition = condition

## *3 - Processus de création des vecteurs features de quelques villes*

### *3-1- Choix des villes*

In [6]:
cities = {'LosAngeles': [33.700615, 34.353627, -118.683511, -118.074559], 
           'Houston': [29.497907,30.129003,-95.797178,-94.988191],
           'Austin': [30.079327, 30.596764,-97.968881,-97.504838],
           'Dallas': [32.559567,33.083278,-97.036586,-96.428928],
           'Charlotte': [34.970168,35.423667,-81.060925,-80.622687],
           'Atlanta': [33.612410,33.916999,-84.575600,-84.231911]}

time_zones = {'Houston':'US/Central', 'Charlotte':'US/Eastern', 'Dallas':'US/Central',
              'Atlanta':'US/Eastern', 'Austin':'US/Central', 'LosAngeles':'US/Pacific'}

# time interval to sample data for 
start = datetime(2018, 6, 1)
finish   = datetime(2018, 9, 2)

begin = datetime.strptime('2018-06-01 00:00:00', '%Y-%m-%d %H:%M:%S')
end   = datetime.strptime('2018-08-31 23:59:59', '%Y-%m-%d %H:%M:%S')

### *3-2- Définissons UTC comme fuseau horaire par défaut pour chaque événement de trafic*

##### *On crée un dictionnaire df_city2incidents qui contient les clés suivantes :* 
*a - EventId*

*b - Type*

*c - LocationLat*

*d - LocationLng*

*e - StartTime(UTC)*

*f - EndTime(UTC)*

In [17]:
## converting time from UTC to local time zone for each city 
path = 'data/temporary/' #create this temporary directory inside the '/data' folder
df_city2incidents = {}
for c in cities:
    incidents = []
    z = time_zones[c]
    
    with open(path + 'TW_EVENT_{}_20180601_20180609.csv'.format(c), 'r') as file:
        header = False
        for line in file:
            if not header:
                header = True
                continue
            parts = line.replace('\r', '').replace('\n', '').split(',')
            
            ds = datetime.strptime(parts[6].replace('T',' '), '%Y-%m-%d %H:%M:%S')
            ds = ds.replace(tzinfo=pytz.utc)
            ds = ds.astimezone(pytz.timezone(z))
            
            de = datetime.strptime(parts[7].replace('T',' '), '%Y-%m-%d %H:%M:%S')
            de = de.replace(tzinfo=pytz.utc)
            de = de.astimezone(pytz.timezone(z))
            
            v = [parts[0], parts[2], float(parts[9]), float(parts[10]), ds, de]            
            incidents.append(v)
            
    df_city2incidents[c] = incidents
    print('TW_EVENT', c, len(incidents))

TW_EVENT LosAngeles 90961
TW_EVENT Houston 45463
TW_EVENT Austin 20438
TW_EVENT Dallas 30543
TW_EVENT Charlotte 18592
TW_EVENT Atlanta 23943


### *3-3- Construisons des vecteurs de fonction paires City-Geohash*

In [18]:
zone_to_be = {}

for z in ['US/Eastern', 'US/Central', 'US/Mountain', 'US/Pacific']:
    t_begin = begin.replace(tzinfo=pytz.timezone(z))
    t_end   = end.replace(tzinfo=pytz.timezone(z))
    zone_to_be[z] = [t_begin, t_end]

name_conversion = {'Broken-Vehicle':'BrokenVehicle', 'Flow-Incident': 'FlowIncident', 'Lane-Blocked':'RoadBlocked'}

#### *3-4- Créons des dictionnaires city_to_geohashes, geocode_to_airport et aiport_to_timezone qui contiennent respectivement les dictionaires des villes, le code des aeroports et  le code des aeroports et les timeZone*

In [19]:
def return_interval_index(time_stamp, start, end):
    if time_stamp < start or time_stamp>end: 
        return -1
    index = int(((time_stamp - start).days*24*60 + (time_stamp-start).seconds/60)/15)
    return index

In [20]:
diff = int(((end - begin).days*24*60 + (end-begin).seconds/60)/15) # total_minutes/15 ==> number of 15 minutes intervals

path = 'data/temporary/'
city_to_geohashes = {}
for c in cities: city_to_geohashes[c] = {}

start_timestamp = time.time()
ccnntt = 0

geocode_to_airport = {}
aiport_to_timezone = {}

for c in cities:
    z = time_zones[c]
    
    # add map-quest data
    with open(path + 'TW_EVENT_{}_20180601_20180609.csv'.format(c), 'r') as file:
        header = False
        for line in file:
            if not header:
                header = True
                continue
            parts = line.replace('\r', '').replace('\n', '').split(',')
            
            ds = datetime.strptime(parts[6].replace('T',' '), '%Y-%m-%d %H:%M:%S')
            ds = ds.replace(tzinfo=pytz.utc)
            ds = ds.astimezone(pytz.timezone(z))
            s_interval = return_interval_index(ds, zone_to_be[z][0], zone_to_be[z][1])
            if s_interval==-1: continue
                
            de = datetime.strptime(parts[7].replace('T',' '), '%Y-%m-%d %H:%M:%S')
            de = de.replace(tzinfo=pytz.utc)
            de = de.astimezone(pytz.timezone(z))
            e_interval = return_interval_index(de, zone_to_be[z][0], zone_to_be[z][1])
            if e_interval == -1: 
              e_interval = diff-1    
            
            start_gh = gh.encode(float(parts[9]), float(parts[10]), precision=geohash_prec)
            intervals = []
            if start_gh not in city_to_geohashes[c]:
                for i in range(diff): 
                    intervals.append({'Construction':0, 'Congestion':0, 'Accident':0, 'FlowIncident':0, 'Event':0, 
                                      'BrokenVehicle':0, 'RoadBlocked':0, 'Other':0})
            else:
                intervals = city_to_geohashes[c][start_gh]
            
            if parts[2] in name_conversion:
                tp = name_conversion[parts[2]]
            else: 
                tp = parts[2].split('-')[0]
                
            for i in range(s_interval, e_interval+1):                
                v = intervals[i]
                if tp in v: 
                  v[tp] = v[tp] + 1
                else: 
                  v['Other'] = v['Other'] + 1
                intervals[i] = v
                
                if tp == 'Accident': break # unlike other types of traffic events, 
                
            city_to_geohashes[c][start_gh] = intervals
            
            ap = parts[12]
            if len(ap) > 3:
                if start_gh not in geocode_to_airport:
                    geocode_to_airport[start_gh] = set([ap])
                else:
                    st = geocode_to_airport[start_gh]
                    st.add(ap)
                    geocode_to_airport[start_gh] = st
                aiport_to_timezone[ap] = z
  
    
    print('Done with {} in {:.1f} sec! there are {} geohashes with data!'.format(c, 
                                time.time()-start_timestamp, len(city_to_geohashes[c])))
    start_timestamp = time.time()
            

Done with LosAngeles in 9.6 sec! there are 156 geohashes with data!
Done with Houston in 6.1 sec! there are 249 geohashes with data!
Done with Austin in 3.7 sec! there are 130 geohashes with data!
Done with Dallas in 6.0 sec! there are 164 geohashes with data!
Done with Charlotte in 3.4 sec! there are 114 geohashes with data!
Done with Atlanta in 4.1 sec! there are 68 geohashes with data!


###### 3-5- Utilisons les Datasets de Sample_Weather, pour créer un dictionnaire de données des codes des aeroports

In [36]:
df = pd.read_csv("data/Sample_Weather/KATL.csv")
df.tail()

,Airport,Date,Hour,Temp,WindChill,Humd,Pressure,Visib,WindSpeed,WindDir,Precipitation,Events,Conditions
35642,KATL,2019-6-30,7:52 PM,86.0,86.0,55.0,28.92,10.0,9.0,WSW,0.0,NaN,Mostly Cloudy
35643,KATL,2019-6-30,8:52 PM,83.0,83.0,63.0,28.91,10.0,8.0,WSW,0.0,NaN,Mostly Cloudy
35644,KATL,2019-6-30,9:52 PM,81.0,81.0,67.0,28.92,10.0,10.0,W,0.0,NaN,Partly Cloudy
35645,KATL,2019-6-30,10:52 PM,79.0,79.0,69.0,28.95,10.0,8.0,W,0.0,NaN,Partly Cloudy
35646,KATL,2019-6-30,11:52 PM,77.0,77.0,76.0,28.97,10.0,7.0,W,0.0,NaN,Fair


In [22]:
# load and sort relevant weather data
airports_to_observations = {}
for g in geocode_to_airport:
    aps = geocode_to_airport[g]
    for a in aps:
        if a not in airports_to_observations:
            airports_to_observations[a] = []

print('{} airports to collect data for!'.format(len(airports_to_observations)))
            
w_path = 'data/Sample_Weather/' # this directory contains weather observation records for each airport
# a sample data file can be find in '/data' directory (Sample_Weather.tar.gz)
airport_to_data = {}
for ap in airports_to_observations:
    data = []
    z = aiport_to_timezone[ap]
    print('Airport {}'.format(ap))
    header = ''
    with open(w_path + ap + '.csv', 'r') as file:
        for line in file:
            if 'Airport' in line: 
                header = line.replace('\r','').replace('\n','').replace(',Hour','')
                continue
            parts = line.replace('\r', '').replace('\n', '').split(',')
            try:
                w = weather(parts[1] + ' ' + parts[2].split(' ')[0] + ':00 ' + parts[2].split(' ')[1], parts[3], parts[4], 
                           parts[5], parts[6], parts[7], parts[8], parts[9], parts[10], parts[11], parts[12], z)   
                data.append(w)
            except:
                continue
    data.sort(key=lambda x:x.date)
    airport_to_data[ap] = data
    
print('\nData for {} airport stations is loaded!'.format(len(airport_to_data)))

51 airports to collect data for!
Airport KSLI
Airport KLGB
Airport KLAX
Airport KHHR
Airport KSMO
Airport KVNY
Airport KBUR
Airport KTOA
Airport KCQT
Airport KFUL
Airport KWHP
Airport KEMT
Airport K3A6
Airport KGXA
Airport KPMD
Airport KMCJ
Airport KHOU
Airport KSGR
Airport KDWH
Airport KTME
Airport KIAH
Airport KEFD
Airport KAXH
Airport KLVJ
Airport K6R3
Airport KAUS
Airport KATT
Airport KRYW
Airport KEDC
Airport KGTU
Airport KRBD
Airport KDAL
Airport KADS
Airport KDFW
Airport KGPM
Airport KLNC
Airport KF46
Airport KHQZ
Airport KTKI
Airport KDTO
Airport KGKY
Airport KCLT
Airport KJQF
Airport KEQY
Airport KUZA
Airport KAKH
Airport KIPJ
Airport KATL
Airport KFTY
Airport KPDK
Airport KMGE

Data for 51 airport stations is loaded!


##### *3-6- Trouvons les aéroports manquants*

In [24]:
for c in city_to_geohashes:
    for g in city_to_geohashes[c]:
        if g not in geocode_to_airport:
            gc = gh.decode_exactly(g)[0:2]
            min_dist = 1000000000
            close_g = ''
            for _g in geocode_to_airport:
                _gc = gh.decode_exactly(_g)[0:2]
                dst = haversine(gc, _gc, 'km')
                if dst < min_dist:
                    min_dist = dst
                    close_g = _g
#            print(g, close_g, min_dist)
            geocode_to_airport[g] = geocode_to_airport[close_g]

In [25]:
city_to_geohashes_to_weather = {}

for c in city_to_geohashes:
    start = time.time()
    geo2weather = {}
    for g in city_to_geohashes[c]:
        w_data = []
        for i in range(len(city_to_geohashes[c][g])):
            w_data.append({'Temperature':[], 'Humidity':[], 'Pressure':[], 'Visibility':[], 'WindSpeed':[], 
                          'Precipitation':[], 'Condition':set(), 'Event':set()})
        # populate weather data
        aps = geocode_to_airport[g]
        for a in aps:
            z = aiport_to_timezone[a]
            a_w_data = airport_to_data[a]
            prev = 0
            for a_w_d in a_w_data:
                idx = return_interval_index(a_w_d.date, zone_to_be[z][0], zone_to_be[z][1])
                if idx >-1:
                    for i in range(prev, min(idx+1, len(w_data))):
                        _w = w_data[i]
                        
                        _tmp = _w['Temperature']
                        if a_w_d.temp > -1000:
                            _tmp.append(a_w_d.temp)
                            _w['Temperature'] = _tmp
                        
                        _hmd = _w['Humidity']
                        if a_w_d.humid > -1000:
                            _hmd.append(a_w_d.humid)
                            _w['Humidity'] = _hmd
                        
                        _prs = _w['Pressure']
                        if a_w_d.pressure > -1000:
                            _prs.append(a_w_d.pressure)
                            _w['Pressure'] = _prs
                        
                        _vis = _w['Visibility']
                        if a_w_d.visib > -1000:
                            _vis.append(a_w_d.visib)
                            _w['Visibility'] = _vis
                            
                        _wspd = _w['WindSpeed']
                        if a_w_d.windspeed > -1000:
                            _wspd.append(a_w_d.windspeed)
                            _w['WindSpeed'] = _wspd
                            
                        _precip = _w['Precipitation']
                        if a_w_d.precipitation > -1000:
                            _precip.append(a_w_d.precipitation)
                            _w['Precipitation'] = _precip
                            
                        _cond = _w['Condition']
                        _cond.add(a_w_d.condition)
                        _w['Condition'] = _cond
                        
                        _evnt = _w['Event']
                        _evnt.add(a_w_d.events)
                        _w['Event'] = _evnt
                        
                        w_data[i] = _w
                        
                    prev = idx+1
                                                
            
        geo2weather[g] = w_data
    city_to_geohashes_to_weather[c] = geo2weather
    print('Done with {} in {:.1f} sec!'.format(c, time.time()-start))

Done with LosAngeles in 26.7 sec!
Done with Houston in 50.1 sec!
Done with Austin in 45.3 sec!
Done with Dallas in 88.7 sec!
Done with Charlotte in 95.0 sec!
Done with Atlanta in 8.8 sec!


In [30]:
print(_w['Condition'])

{'Scattered Clouds', 'Fair'}


#### *3-7- Ajouter le Dataset Daylight*

In [ ]:
class dayLight:
    sunrise = []
    sunset = []
    def __init__(self, sunrise, sunset):
        self.sunrise = sunrise
        self.sunset = sunset
        
def return_time(x):
    try:
        h = int(x.split(':')[0])
        m = int(x.split(':')[1].split(' ')[0])
        if 'pm' in x and h < 12: h = h + 12
        return [h,m]
    except: return [0,0]

    
def returnDayLight(city, state, dt):
    sc = city + '-' + state
    days = city_days_time[sc]
    d = str(dt.year) + '-' + str(dt.month) + '-' + str(dt.day)
    if d in days:
        r = days[d]
        if ((dt.hour>r.sunrise[0] and dt.hour<r.sunset[0]) or
            (dt.hour>=r.sunrise[0] and dt.minute>=r.sunrise[1] and dt.hour<r.sunset[0]) or
            (dt.hour>r.sunrise[0] and dt.hour<=r.sunset[0] and dt.minute<r.sunset[1]) or 
            (dt.hour>=r.sunrise[0] and dt.minute>=r.sunrise[1] and dt.hour<=r.sunset[0] and dt.minute<r.sunset[1])):
            return '1'
        else: return '0'

In [ ]:
city_days_time = {}

days = {}
city = ''
with open('data/sample_daylight.csv', 'r') as file: # you find daylight data for the selected 6 cities in this file
    for ln in file.readlines():
        parts = ln.replace('\r','').replace('\n','').split(',')

        if parts[0] != city:
            if len(city) > 0: 
                if city in city_days_time:
                    _days = city_days_time[city]
                    for _d in _days: days[_d] = _days[_d]
                city_days_time[city] = days

            city = parts[0]
            days = {}

        sunrise = return_time(parts[2])
        sunset  = return_time(parts[3])
        dl = dayLight(sunrise, sunset)
        days[parts[1]] = dl

if city in city_days_time:
    _days = city_days_time[city]
    for _d in _days: days[_d] = _days[_d]
city_days_time[city] = days


print('Successfully loaded daylight data for {} cities!'.format(len(city_days_time)))

#### *3-8- créons un dictionnaires de daylight des indexes des villes*

In [ ]:
# pre-load daylight mapping for different cities
city_to_index_to_daylight = {}
states = {'Houston':'TX', 'Charlotte':'NC', 'Dallas':'TX', 'Atlanta':'GA', 'Austin':'TX', 'LosAngeles':'CA'}
for c in cities:
    d_begin = begin.replace(tzinfo=pytz.timezone(time_zones[c]))
    d_end   = end.replace(tzinfo=pytz.timezone(time_zones[c]))
    index_to_daylight = {}
    index = 0
    while(d_begin < d_end):
        dl = returnDayLight(c, states[c], d_begin)
        index_to_daylight[index] = dl
        index += 1
        d_begin += timedelta(seconds=15*60)
    city_to_index_to_daylight[c] = index_to_daylight
    

#### *3-9- Enfin créons des vecteurs features des villes choisies et récupérons les dataset créés*

In [ ]:
# map each time-step to hour of day and day of the week; this should be consistent across different time-zones!
timestep_to_dow_hod = {}
d_begin = begin.replace(tzinfo=pytz.utc)
d_end   = end.replace(tzinfo=pytz.utc)
index = 0

while(d_begin < d_end):
    dow = d_begin.weekday()
    hod = d_begin.hour    
    timestep_to_dow_hod[index] = [dow, hod]
    
    d_begin += timedelta(seconds=15*60)    
    index += 1

In [40]:
traffic_tags = ['Accident', 'BrokenVehicle', 'Congestion', 'Construction', 'Event', 'FlowIncident', 'Other', 'RoadBlocked']
weather_tags = ['Condition', 'Event', 'Humidity', 'Precipitation', 'Pressure', 'Temperature', 'Visibility', 'WindSpeed']
poi_tags = []
start = time.time()
condition_tags = set()

for c in city_to_geohashes:
    # creating vector for each reion (geohash) during a 15 minutes time interval. Such vector contains time, traffic, and weather attributes. 
    writer = open('data/vectors/{}_geo2vec_{}-{}.csv'.format(c, str(begin.year)+str(begin.month)+str(begin.day),
                                                        str(end.year)+str(end.month)+str(end.day)), 'w')
    writer.write('Geohash,TimeStep,DOW,HOD,DayLight,T-Accident,T-BrokenVehicle,T-Congestion,T-Construction,'\
        'T-Event,T-FlowIncident,T-Other,T-RoadBlocked,W-Humidity,W-Precipitation,W-Pressure,'\
        'W-Temperature,W-Visibility,W-WindSpeed,W-Rain,W-Snow,W-Fog,W-Hail\n')
    
    traffic = city_to_geohashes[c]
    weather = city_to_geohashes_to_weather[c]        
    for g in traffic:
        vectors = []
        for i in range(len(traffic[g])):
            v = []
            for t in traffic_tags: v.append(traffic[g][i][t])
            v_w = [0,0,0,0] # for rain, snow, fog, and hail
            for w in weather_tags:
                if w=='Condition' or w=='Event':      
                    _tgs = weather[g][i][w]
                    for _tg in _tgs: 
                        if 'rain' in _tg.lower() or 'drizzle' in _tg.lower() or 'thunderstorm' in _tg.lower(): v_w[0] = 1
                        elif 'snow' in _tg.lower(): v_w[1] = 1
                        elif 'fog' in _tg.lower() or 'haze' in _tg.lower() or 'mist' in _tg.lower() or 'smoke' in _tg.lower(): v_w[2] = 1
                        elif 'hail' in _tg.lower() or 'ice pellets' in _tg.lower(): v_w[3] = 1                            
                elif len(weather[g][i][w]) == 0: v.append(0)
                else: v.append(np.mean(weather[g][i][w]))
            for _v_w in v_w: v.append(_v_w)
            vectors.append(v)
        
        for i in range(len(vectors)):
            v = vectors[i]
            v = [str(v[j]) for j in range(len(v))]
            v = ','.join(v)
            writer.write(g + ',' + str(i) + ',' + str(timestep_to_dow_hod[i][0]) + ',' + str(timestep_to_dow_hod[i][1]) 
                         + ',' + city_to_index_to_daylight[c][i] + ',' + v + '\n')
            
    writer.close()
    print('Done with {} in {:.1f} sec! #vectors {}!'.format(c, time.time()-start, len(traffic)*len(vectors)))
    start = time.time()

Done with LosAngeles in 113.5 sec! #vectors 1377636!
Done with Houston in 179.3 sec! #vectors 2198919!
Done with Austin in 94.2 sec! #vectors 1148030!
Done with Dallas in 119.7 sec! #vectors 1448284!
Done with Charlotte in 80.5 sec! #vectors 1006734!
Done with Atlanta in 48.0 sec! #vectors 600508!


### !!! Fin de Generation des vecteurs features, pour la suite, voir le fichier !!!